# Retired 12/27

In [23]:
import pygame
import random
import matplotlib.pyplot as plt
import math as ma
import numpy as np
import scipy as sp
from scipy.spatial.transform import Rotation as R

In [2]:
np.set_printoptions(precision=3, sign=' ')

# Adds a new face and color pair to a pair of lists
def newFace(f,c,fL,cL):
    fL = np.concatenate((fL,f),axis=0)
    cL = np.concatenate((cL,c),axis=0)
    return fL,cL

# Uses a vector and origin to see which item (from flist) is hit. 
def rayNumbers(v,O,rst):
    t=rst[:,2]
    B=rst[:,1]-rst[:,2] #The order here is basically determined by wanting BxC to match the face vector &
    C=rst[:,0]-rst[:,2] # wanting both vectors to start at t (although any point would do).
    l=O-rst[:,2]
    U=np.stack((B,C,np.broadcast_to(-v,B.shape)),axis=1)
    # print(U)
    try:
        N=np.linalg.inv(U)
    except:
        print('Singular Matrix')
        N=np.broadcast_to(np.eye(3,3),U.shape)
    u=np.matmul(l[:,None,:],N)[:,0,:] # Not sure why matmul has to go l,N instead of N,l
    return u

def rayNumbersV(V,O,rst):
    indexes = []
    for v in V:
        #print(v)
        rN = rayNumbers(v,O,rst)
        rNBool = np.any(np.vstack((0>rN[:,0],0>rN[:,1],1<rN[:,0]+rN[:,1],0>=rN[:,2])),axis=0)
        rN[rNBool,2]=np.inf
        indexes.append(np.argmin(rN[:,2]))
    return indexes

def raysOfVision(O,D):
    '''
    Uses an orientation to return a number of rays which will be used to construct the Vision
    @param O : The orientation of the viewer
    @param D : A number characterizing the number of rays to be produced
    @return : A np.array of rays
    '''
    assert isinstance(O, sp.spatial.transform.Rotation)
    return None

def normV(v):
    return v / np.linalg.norm(v, axis=len(v.shape)-1, keepdims=True)

def newFace(f,c,fL,cL):
    fL = np.concatenate((fL,f),axis=0)
    cL = np.concatenate((cL,c),axis=0)
    return fL,cL

def outAround(out,around):
    '''
    Takes 0,0,1 rotates it round the x axis (down) by the out angle and then rotates it around the z axis by the around angle
    only works with one pair of angles at the momen
    '''
    return R.from_euler('xz', [out,around], degrees=True).apply(np.array([0,0,1]))

def outAroundInv(v):
    '''
    Inverts outAround
    '''
    if len(v.shape)==1:
        return np.arccos(v[2])*180/np.pi, (np.arctan2(v[1],v[0])*180/np.pi)+90
    elif len(v.shape)==2:
        return np.arccos(v[:,2])*180/np.pi, (np.arctan2(v[:,1],v[:,0])*180/np.pi)+90
    else:
        raise Exception("This function can only handle lists of vectors") 

def octahedron(center,r,rot,lowColor=3,highColor=7):
    #center=np.array([0,0,0])
    #r=3
    #rot=R.from_rotvec(np.array([.1, .1, .1]))
    offset=np.transpose((np.stack((np.eye(3),np.eye(3)*-1),axis=2)).reshape(3,6))
    corners=r*offset+center
    #print(offset)
    sideSel=np.array([[1,0,1,0,1,0],[1,0,1,0,0,1],[1,0,0,1,1,0],[0,1,1,0,1,0],[0,1,1,0,0,1],[1,0,0,1,0,1],[0,1,0,1,1,0],[0,1,0,1,0,1]],dtype=bool)
    sides=np.zeros((1,3,3))
    for i in sideSel:
        sides=np.append(sides,offset[None,i],axis=0)
    sides=sides[1:]
    #print(sides)
    #print(rot.apply(sides.reshape(24,3)).reshape(8,3,3))
    sidesR=rot.apply(sides.reshape(24,3)).reshape(8,3,3)*r+center
    lC,hC=lowColor,highColor #low and high color
    colors=(np.sum(sides,axis=2)+1)*(hC-lC)/2+lC
    #print(colors.shape)
    return sidesR,colors
def tickMark(loc,color,orientation=0,scale=1):
    side=np.array([[0,0,0],[1,0,0],[0,1,0]])
    if orientation!=0:
        side=orientation.apply(side)
    return side, np.array(color)

In [3]:
octahedron(np.array([0.1,0.2,0.3]),30,R.from_rotvec(np.array([.1, .1, .1])))

(array([[[ 29.801,   3.335,  -2.535],
         [ -2.735,  29.901,   3.435],
         [  3.235,  -2.635,  30.001]],
 
        [[ 29.801,   3.335,  -2.535],
         [ -2.735,  29.901,   3.435],
         [ -3.035,   3.035, -29.401]],
 
        [[ 29.801,   3.335,  -2.535],
         [  2.935, -29.501,  -2.835],
         [  3.235,  -2.635,  30.001]],
 
        [[-29.601,  -2.935,   3.135],
         [ -2.735,  29.901,   3.435],
         [  3.235,  -2.635,  30.001]],
 
        [[-29.601,  -2.935,   3.135],
         [ -2.735,  29.901,   3.435],
         [ -3.035,   3.035, -29.401]],
 
        [[ 29.801,   3.335,  -2.535],
         [  2.935, -29.501,  -2.835],
         [ -3.035,   3.035, -29.401]],
 
        [[-29.601,  -2.935,   3.135],
         [  2.935, -29.501,  -2.835],
         [  3.235,  -2.635,  30.001]],
 
        [[-29.601,  -2.935,   3.135],
         [  2.935, -29.501,  -2.835],
         [ -3.035,   3.035, -29.401]]]),
 array([[ 7.,  7.,  7.],
        [ 7.,  7.,  3.],
        [ 7., 

In [4]:
center=np.array([0,0,0])
r=3
rot=R.from_rotvec(np.array([.1, .1, .1]))
offset=np.transpose((np.stack((np.eye(3),np.eye(3)*-1),axis=2)).reshape(3,6))
corners=r*offset+center
#print(offset)
sideSel=np.array([[1,0,1,0,1,0],[1,0,1,0,0,1],[1,0,0,1,1,0],[0,1,1,0,1,0],[0,1,1,0,0,1],[1,0,0,1,0,1],[0,1,0,1,1,0],[0,1,0,1,0,1]],dtype=bool)
sides=np.zeros((1,3,3))
for i in sideSel:
    sides=np.append(sides,offset[None,i],axis=0)
sides=sides[1:]
#print(sides)
#print(rot.apply(sides.reshape(24,3)).reshape(8,3,3))
#print(rot.apply(sides.reshape(24,3)).reshape(8,3,3)*r+center)
lC,hC=1,8 #low and high color
colors=(np.sum(sides,axis=2)+1)*(hC-lC)/2+lC
#print(colors.shape)

In [5]:
out=np.array([30,30])
around=np.array([30,40])
print(outAround(out,around))
print(outAroundInv(outAround(out,around)))

[[ 0.25  -0.433  0.866]
 [ 0.321 -0.383  0.866]]
(array([ 30.,  30.]), array([ 30.,  40.]))


In [6]:
listN = 5
fList,cList=octahedron(np.array([0.1,0.2,0.3]),30,R.from_rotvec(np.array([.1, .1, .1])))
nf = np.random.randint(-5, 6, (listN,3,3), dtype=int)
nc = np.random.randint(0, 10, (listN,3), dtype=int)
fList,cList = newFace(nf,nc,fList,cList)
V = np.array([1,1,0.5])
V2 = np.array([[1,0,0],[1,0.5,0],[1,1,0],[1,1,0.5],[1,1,1]])
O = np.array([0,0,0])

In [7]:
rN = rayNumbers(V,O,fList)
print(rN)
rNBool = np.any(np.vstack((0>rN[:,0],0>rN[:,1],1<rN[:,0]+rN[:,1],0>=rN[:,2])),axis=0)
outtable = np.concatenate((rN,rNBool[:,None],rN[:,None,2]),axis=1)
outtable[rNBool,4]=np.inf
print("   0<  |   0<  |   1>  |   0<  |Failed?|  dist |")
for i in outtable:
    print("%7.3f %7.3f %7.3f %7.3f %7.0f %7.3f"%(i[0],i[1],i[0]+i[1],i[2],i[3],i[4]))
nearIndex = np.argmin(outtable[:,4])
print(nearIndex)
print(cList[nearIndex])

[[  0.379   0.424  12.24 ]
 [  0.629   0.7    20.149]
 [ -1.569   1.739  49.894]
 [  2.362  -2.616  75.013]
 [ -1.567   1.726 -49.364]
 [  2.312  -2.55  -72.957]
 [  0.643   0.705 -20.119]
 [  0.379   0.414 -11.76 ]
 [ -0.267   0.733  -1.2  ]
 [  0.5     1.      2.   ]
 [  0.313   0.641  -2.844]
 [ -0.324   0.676  -0.176]
 [  1.125  -1.688  -2.375]]
   0<  |   0<  |   1>  |   0<  |Failed?|  dist |
  0.379   0.424   0.804  12.240       0  12.240
  0.629   0.700   1.330  20.149       1     inf
 -1.569   1.739   0.170  49.894       1     inf
  2.362  -2.616  -0.253  75.013       1     inf
 -1.567   1.726   0.159 -49.364       1     inf
  2.312  -2.550  -0.238 -72.957       1     inf
  0.643   0.705   1.348 -20.119       1     inf
  0.379   0.414   0.792 -11.760       1     inf
 -0.267   0.733   0.467  -1.200       1     inf
  0.500   1.000   1.500   2.000       1     inf
  0.313   0.641   0.953  -2.844       1     inf
 -0.324   0.676   0.353  -0.176       1     inf
  1.125  -1.688  -0.563

In [8]:
nearIndex2=rayNumbersV(V2,O,fList)
print(nearIndex2)
print(cList[nearIndex2])

Singular Matrix
Singular Matrix
Singular Matrix
[0, 0, 0, 0, 0]
[[ 7.  7.  7.]
 [ 7.  7.  7.]
 [ 7.  7.  7.]
 [ 7.  7.  7.]
 [ 7.  7.  7.]]


In [9]:
rN = rayNumbers(np.array([1,1,1]),O,fList)
rNBool = np.any(np.vstack((0>rN[:,0],0>rN[:,1],1<rN[:,0]+rN[:,1],0>=rN[:,2])),axis=0)
outtable = np.concatenate((rN,rNBool[:,None],rN[:,None,2]),axis=1)
outtable[rNBool,4]=np.inf
print("   0<  |   0<  |   1>  |   0<  |Failed?|  dist |")
for i in outtable:
    print("%7.3f %7.3f %7.3f %7.3f %7.0f %7.3f"%(i[0],i[1],i[0]+i[1],i[2],i[3],i[4]))
nearIndex = np.argmin(outtable[:,4])
print(nearIndex)
print(cList[nearIndex])

   0<  |   0<  |   1>  |   0<  |Failed?|  dist |
  0.333   0.337   0.670  10.200       0  10.200
  0.993   0.998   1.991  30.023       1     inf
 -0.998   1.002   0.004  30.160       1     inf
  1.007  -1.011  -0.004  30.417       1     inf
 -1.002   0.998  -0.004 -29.840       1     inf
  0.993  -0.989   0.004 -29.583       1     inf
  1.007   1.002   2.009 -29.977       1     inf
  0.334   0.330   0.664  -9.800       1     inf
 -1.167   1.333   0.167  -3.000       1     inf
  0.500   0.667   1.167   1.000       1     inf
  0.191   0.489   0.681  -1.936       1     inf
 -0.320   0.680   0.360  -0.120       1     inf
  0.909  -1.364  -0.455  -1.727       1     inf
0
[ 7.  7.  7.]


In [10]:
pzh=np.array([0,0,1])
print(R.from_euler('xz', [-45,180], degrees=True).apply(pzh))
print(R.from_euler('xz', [1,90], degrees=True).apply(pzh))
print(R.from_euler('xz', [10,45], degrees=True).apply(pzh))
print(R.from_euler('xz', [45,0], degrees=True).apply(pzh))
print(R.from_euler('xz', [45,45], degrees=True).apply(pzh))
print(R.from_euler('xz', [60,60], degrees=True).apply(pzh))
print(R.from_euler('xz', [90,0], degrees=True).apply(pzh))
print(R.from_euler('xz', [90,90], degrees=True).apply(pzh))
print(R.from_euler('xz', [[10,45],[-10,45]], degrees=True).apply(pzh))

[-8.660e-17 -7.071e-01  7.071e-01]
[ 0.017  0.     1.   ]
[ 0.123 -0.123  0.985]
[ 0.    -0.707  0.707]
[ 0.5   -0.5    0.707]
[ 0.75  -0.433  0.5  ]
[ 0. -1.  0.]
[ 1.  0.  0.]
[[ 0.123 -0.123  0.985]
 [-0.123  0.123  0.985]]


In [11]:
v45=outAround(45,270)
print(v45)
around = (np.arctan2(v45[1],v45[0])*180/np.pi)+90
out = np.arccos(v45[2])*180/np.pi
print(out,around)

[-7.071e-01  1.110e-16  7.071e-01]
45.00000000000001 270.0


In [12]:
# Goal: Take points from a surface and create a method which repositions it relative to the player
# probably something like (point+player location)*player rotation
shape=np.array([[2,1,0],[1,1,3],[1,2,1]])
O=np.array([0,0,0]) #Player location
# right off the bat this would appear as [[1,0,2],[1,3,1],[2,1,1]]
playerOrient = R.from_rotvec(-2/3*np.pi * normV(np.array([1,1,1])))
print(playerOrient.apply(shape-O)) # Good start

[[ 1.00e+00  1.11e-16  2.00e+00]
 [ 1.00e+00  3.00e+00  1.00e+00]
 [ 2.00e+00  1.00e+00  1.00e+00]]


In [13]:
O=np.array([-1,0,0]) #Player location
# right off the bat this would appear as [[1,0,3],[1,3,2],[2,1,2]]
playerOrient = R.from_rotvec(-2/3*np.pi * normV(np.array([1,1,1])))
print(playerOrient.apply(shape-O)) # Good start

[[ 1.00e+00  2.22e-16  3.00e+00]
 [ 1.00e+00  3.00e+00  2.00e+00]
 [ 2.00e+00  1.00e+00  2.00e+00]]


In [14]:
rotx10=R.from_euler('x', 10, degrees=True) # hopefully this will be a rotation abote the player's x axis looking up a bit
print((rotx10 * playerOrient).apply(shape-O)) # Yes?

[[ 1.    -0.521  2.954]
 [ 1.     2.607  2.491]
 [ 2.     0.638  2.143]]


In [15]:
roty10=R.from_euler('y', 10, degrees=True) # hopefully this will be a rotation abote the player's y axis looking left a bit
print((roty10 * playerOrient).apply(shape)) # Yes?
# seems things follow the right hand curl rule

[[ 1.332e+00  1.110e-16  1.796e+00]
 [ 1.158e+00  3.000e+00  8.112e-01]
 [ 2.143e+00  1.000e+00  6.375e-01]]


In [16]:
rotx90=R.from_euler('x', 90, degrees=True)
roty90=R.from_euler('y', 90, degrees=True)
print((rotx90*playerOrient).apply(shape)) 
print((roty90*rotx90*playerOrient).apply(shape)) 
print((rotx90*roty90*playerOrient).apply(shape)) 

[[ 1.00e+00 -2.00e+00  7.85e-17]
 [ 1.00e+00 -1.00e+00  3.00e+00]
 [ 2.00e+00 -1.00e+00  1.00e+00]]
[[ 1.11e-16 -2.00e+00 -1.00e+00]
 [ 3.00e+00 -1.00e+00 -1.00e+00]
 [ 1.00e+00 -1.00e+00 -2.00e+00]]
[[ 2.00e+00  1.00e+00  1.11e-16]
 [ 1.00e+00  1.00e+00  3.00e+00]
 [ 1.00e+00  2.00e+00  1.00e+00]]


In [17]:
nFaces=fList.shape[0]
fListFlat=fList.reshape(nFaces*3,3)
fListP=(roty90*rotx90*playerOrient).apply(fListFlat).reshape(nFaces,3,3)

In [18]:
playerOrient.as_matrix()

array([[-1.11e-16,  1.00e+00,  1.11e-16],
       [ 1.11e-16, -1.11e-16,  1.00e+00],
       [ 1.00e+00,  1.11e-16, -1.11e-16]])

In [19]:
pLoc = np.array([1,1,1])
pLoc=playerOrient.apply(np.array([1,0,0]))+pLoc

In [20]:
np.array([1,0,0])

array([1, 0, 0])

In [21]:
def outAround2Screen(s,oA):
    SCREEN_DIMENSION=s.get_size()
    sCx=SCREEN_DIMENSION[0]/2
    sCy=SCREEN_DIMENSION[1]/2
    sRadius=min(sCx,sCy)
    oRadius=oA[0]*sRadius/180
    x=oRadius*ma.sin(ma.radians(oA[1]))
    y=oRadius*ma.cos(ma.radians(oA[1]))
    return sCx+x,sCy+y
def outAround(out,around):
    '''
    Takes 0,0,1 rotates it round the x axis (down) by the out angle and then rotates it around the z axis by the around angle
    only works with one pair of angles at the momen
    '''
    return R.from_euler('xz', [out,around], degrees=True).apply(np.array([0,0,1]))
def eyeVec2persp(vec):
    '''
    Perspective should refer to the normalized version of what goes onto the screen. Screen should refer to what gets put onto the screen
    Takes perspective coordintes and returns the eye vector corresponding to that perspective coordinate
    '''
    coords=np.reshape(coord,(int(coord.size/2),2))
    outDist=np.linalg.norm(coords,axis=1)
    outAngle=np.arcsin(outDist)*2
    print(coords)
    print(outDist)
    print(np.degrees(outAngle))
    # next we need to convert the 
def pixelCircle(r,centers=False):
    '''
    Returns pixels a distance r from center is at coordiate point 0,0 and pixels are assumed to be 1 unit wide.
    The closest pixel to the center has the center (0.5,0.5) It is assumed that this level of detail will matter some day.
    '''
    r2=r*r
    pixels=np.array([[r*ma.cos(ma.pi/4)-0.5,r*ma.sin(ma.pi/4)-0.5]],dtype=int)
    y = pixels[0,1]
    x = pixels[0,0]
    cenStack=np.stack((np.arange(0,x),np.arange(0,y))).T
    # print(x,y)
    while(y>=0 and x<r):
        # print(x+1,y,ma.pow(x+1+0.5,2)+ma.pow(y+0.5,2))
        if ma.pow(x+1+0.5,2)+ma.pow(y+0.5,2)<r2 :
            x+=1
        else :
            y-=1
        newStack = np.stack((np.arange(x-y,x+1),np.arange(0,y+1))).T
        pixels=np.concatenate((pixels,newStack),axis=0)
        
    # print(np.flip(pixels[1:,:],axis=1))
    pixels=np.concatenate((pixels,np.flip(pixels[1:,:],axis=1)),axis=0)
    pixels=np.concatenate((pixels,cenStack),axis=0)
    # rotating the quarter circle
    pixels=pixels+0.5
    pixelX=np.multiply(pixels,np.broadcast_to(np.array([-1,1]),(pixels.shape)))
    pixels=np.concatenate((pixels,pixelX),axis=0)
    pixelY=np.multiply(pixels,np.broadcast_to(np.array([1,-1]),(pixels.shape)))
    pixels=np.concatenate((pixels,pixelY),axis=0)
    if centers:
        return pixels
    else:
        pixels=pixels-0.5
        return(pixels.astype(int))
def persp2eyeVec(coords):
    '''
    Perspective should refer to the normalized version of what goes onto the screen. Screen should refer to what gets put onto the screen
    Takes perspective coordintes and returns the eye vector corresponding to that perspective coordinate
    '''
    # print(coords)
    coords=np.reshape(coord,(int(coord.size/2),2))
    outDist=np.linalg.norm(coords,axis=1)
    outAngle=np.arcsin(outDist)*2
    aroundAngle=np.arctan2(coords[:,1],coords[:,0])+np.pi/2
    print(coords.T)
    print(outDist)
    # next we need to convert the 
    OACoords=np.array([outAngle,aroundAngle]).T
    print(np.degrees(OACoords).T)
    vector=R.from_euler('xz', OACoords, degrees=False).apply(np.array([0,0,1]))
    return vector

In [24]:
pixelCircle(2,True)

array([[ 0.5,  0.5],
       [ 1.5,  0.5],
       [ 0.5,  1.5],
       [-0.5,  0.5],
       [-1.5,  0.5],
       [-0.5,  1.5],
       [ 0.5, -0.5],
       [ 1.5, -0.5],
       [ 0.5, -1.5],
       [-0.5, -0.5],
       [-1.5, -0.5],
       [-0.5, -1.5]])

In [25]:
coord=np.reshape(np.array([.999,0,.71,0,-.3,.2,.1,-.1,-.5,-.2]),(5,2))
coord=np.array([.999,0,.71,0,.38,0,0,0])
coord=pixelCircle(20,True)/20

print(coord.shape)
print(persp2eyeVec(coord).shape)

(1264, 2)
[[ 0.675  0.075  0.125 ... -0.525 -0.575 -0.625]
 [ 0.675  0.025  0.075 ... -0.525 -0.575 -0.625]]
[ 0.955  0.079  0.146 ...  0.742  0.813  0.884]
[[145.336   9.069  16.764 ...  95.883 108.814 124.229]
 [135.    108.435 120.964 ... -45.    -45.    -45.   ]]
(1264, 3)


In [26]:
pC=pixelCircle(2,True)
sRadius=1
sCx=20
sCy=10
print(pC)
print(np.multiply(perspCoords*sRadius,np.broadcast_to(np.array([1,-1]),(perspCoords.shape)))+np.array([sCx,sCy]))

[[ 0.5  0.5]
 [ 1.5  0.5]
 [ 0.5  1.5]
 [-0.5  0.5]
 [-1.5  0.5]
 [-0.5  1.5]
 [ 0.5 -0.5]
 [ 1.5 -0.5]
 [ 0.5 -1.5]
 [-0.5 -0.5]
 [-1.5 -0.5]
 [-0.5 -1.5]]


NameError: name 'perspCoords' is not defined

# The PyGame Starts Here

In [1]:
import pygame
import random
import numpy as np
import scipy as sp
from scipy.spatial.transform import Rotation as R
from pygame.locals import (RLEACCEL,K_ESCAPE,KEYDOWN,QUIT)
from pygame.locals import (K_UP,K_DOWN,K_LEFT,K_RIGHT)
from pygame.locals import (K_q,K_w,K_e,K_a,K_s,K_d,K_z,K_x,K_c)
from pygame.locals import (K_u,K_i,K_o,K_j,K_k,K_l,K_COMMA)
import math as ma
pygame.init()

pygame 2.0.0 (SDL 2.0.12, python 3.7.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


(6, 0)

In [2]:
np.set_printoptions(precision=3, sign=' ')

# Adds a new face and color pair to a pair of lists
def newFace(f,c,fL,cL):
    fL = np.concatenate((fL,f),axis=0)
    cL = np.concatenate((cL,c),axis=0)
    return fL,cL

# Uses a vector and origin to see which item (from flist) is hit. 
def rayNumbers(v,O,rst):
    t=rst[:,2]
    B=rst[:,1]-rst[:,2] #The order here is basically determined by wanting BxC to match the face vector &
    C=rst[:,0]-rst[:,2] # wanting both vectors to start at t (although any point would do).
    l=O-rst[:,2]
    U=np.stack((B,C,np.broadcast_to(-v,B.shape)),axis=1)
    # print(U)
    try:
        N=np.linalg.inv(U)
    except:
        print('Singular Matrix')
        N=np.broadcast_to(np.eye(3,3),U.shape)
    u=np.matmul(l[:,None,:],N)[:,0,:] # Not sure why matmul has to go l,N instead of N,l
    return u

def rayNumbersV(V,O,rst):
    indexes = []
    for v in V:
        #print(v)
        rN = rayNumbers(v,O,rst)
        rNBool = np.any(np.vstack((0>rN[:,0],0>rN[:,1],1<rN[:,0]+rN[:,1],0>=rN[:,2])),axis=0)
        rN[rNBool,2]=np.inf
        indexes.append(np.argmin(rN[:,2]))
    return indexes

def raysOfVision(O,D):
    '''
    Uses an orientation to return a number of rays which will be used to construct the Vision
    @param O : The orientation of the viewer
    @param D : A number characterizing the number of rays to be produced
    @return : A np.array of rays
    '''
    assert isinstance(O, sp.spatial.transform.Rotation)
    return None

def normV(v):
    return v / np.linalg.norm(v, axis=len(v.shape)-1, keepdims=True)

def newFace(f,c,fL,cL):
    fL = np.concatenate((fL,f),axis=0)
    cL = np.concatenate((cL,c),axis=0)
    return fL,cL

def outAround(out,around):
    '''
    Takes 0,0,1 rotates it round the x axis (down) by the out angle and then rotates it around the z axis by the around angle
    only works with one pair of angles at the momen
    '''
    return R.from_euler('xz', [out,around], degrees=True).apply(np.array([0,0,1]))

def outAroundInv(v):
    '''
    Inverts outAround
    '''
    if len(v.shape)==1:
        return np.arccos(v[2])*180/np.pi, (np.arctan2(v[1],v[0])*180/np.pi)+90
    elif len(v.shape)==2:
        return np.arccos(v[:,2])*180/np.pi, (np.arctan2(v[:,1],v[:,0])*180/np.pi)+90
    else:
        raise Exception("This function can only handle lists of vectors") 

def octahedron(center,r,rot,lowColor=3,highColor=7):
    #center=np.array([0,0,0])
    #r=3
    #rot=R.from_rotvec(np.array([.1, .1, .1]))
    offset=np.transpose((np.stack((np.eye(3),np.eye(3)*-1),axis=2)).reshape(3,6))
    corners=r*offset+center
    #print(offset)
    sideSel=np.array([[1,0,1,0,1,0],[1,0,1,0,0,1],[1,0,0,1,1,0],[0,1,1,0,1,0],[0,1,1,0,0,1],[1,0,0,1,0,1],[0,1,0,1,1,0],[0,1,0,1,0,1]],dtype=bool)
    sides=np.zeros((1,3,3))
    for i in sideSel:
        sides=np.append(sides,offset[None,i],axis=0)
    sides=sides[1:]
    #print(sides)
    #print(rot.apply(sides.reshape(24,3)).reshape(8,3,3))
    sidesR=rot.apply(sides.reshape(24,3)).reshape(8,3,3)*r+center
    lC,hC=lowColor,highColor #low and high color
    colors=(np.sum(sides,axis=2)+1)*(hC-lC)/2+lC
    #print(colors.shape)
    return sidesR,colors
def tickMark(loc,color,orientation=0,scale=1):
    side=np.array([[0,0,0],[1,0,0],[0,1,0]])
    if orientation!=0:
        side=orientation.apply(side)
    return side, np.array(color)

In [3]:
# Define a Player object by extending pygame.sprite.Sprite
# The surface drawn on the screen is now an attribute of 'player'
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        self.location=np.array([0,0,0])
        self.orientation = R.from_rotvec(-2/3*np.pi * normV(np.array([1,1,1])))
    
    # Move the sprite based on user keypresses
    def update(self, pressed_keys):
        if pressed_keys[K_w]:
            self.location=self.location+self.orientation.apply(np.array([1,0,0]))
        if pressed_keys[K_s]:
            self.location=self.location-self.orientation.apply(np.array([1,0,0]))
        if pressed_keys[K_a]:
            self.location=self.location-self.orientation.apply(np.array([0,1,0]))
        if pressed_keys[K_d]:
            self.location=self.location+self.orientation.apply(np.array([0,1,0]))
        if pressed_keys[K_q]:
            self.location=self.location+self.orientation.apply(np.array([0,0,1]))
        if pressed_keys[K_e]:
            self.location=self.location-self.orientation.apply(np.array([0,0,1]))
        
        if pressed_keys[K_z]:
            self.location-=np.array([1,0,0])
        if pressed_keys[K_x]:
            self.location=np.array([0,0,0])
        if pressed_keys[K_c]:
            self.location+=np.array([1,0,0])
            
        if pressed_keys[K_i]:
            self.orientation=R.from_euler('x', -10, degrees=True)*self.orientation
        if pressed_keys[K_k]:
            self.orientation=R.from_euler('x', 10, degrees=True)*self.orientation
        if pressed_keys[K_j]:
            self.orientation=R.from_euler('y', 10, degrees=True)*self.orientation
        if pressed_keys[K_l]:
            self.orientation=R.from_euler('y', -10, degrees=True)*self.orientation
        if pressed_keys[K_u]:
            self.orientation=R.from_euler('z', 10, degrees=True)*self.orientation
        if pressed_keys[K_o]:
            self.orientation=R.from_euler('z', -10, degrees=True)*self.orientation
        if pressed_keys[K_COMMA]:
            self.orientation = R.from_rotvec(-2/3*np.pi * normV(np.array([1,1,1])))

In [4]:
def outAround2Screen(s,oA):
    SCREEN_DIMENSION=s.get_size()
    sCx=SCREEN_DIMENSION[0]/2
    sCy=SCREEN_DIMENSION[1]/2
    sRadius=min(sCx,sCy)
    oRadius=oA[0]*sRadius/180
    x=oRadius*ma.sin(ma.radians(oA[1]))
    y=oRadius*ma.cos(ma.radians(oA[1]))
    return sCx+x,sCy+y
def pixelCircle(r):
    '''
    Returns pixels a distance r from center is at coordiate point 0,0 and pixels are assumed to be 1 unit wide.
    The closest pixel to the center has the center (0.5,0.5) It is assumed that this level of detail will matter some day.
    '''
    r2=r*r
    pixels=np.array([[r*ma.cos(ma.pi/4)-0.5,r*ma.sin(ma.pi/4)-0.5]],dtype=int)
    y = pixels[0,1]
    x = pixels[0,0]
    cenStack=np.stack((np.arange(0,x),np.arange(0,y))).T
    while(y>0 and x<r):
        if ma.pow(x+1+0.5,2)+ma.pow(y+0.5,2)<r2 :
            x+=1
        else :
            y-=1
        newStack = np.stack((np.arange(x-y,x+1),np.arange(0,y+1))).T
        pixels=np.concatenate((pixels,newStack),axis=0)
        
    # print(np.flip(pixels[1:,:],axis=1))
    pixels=np.concatenate((pixels,np.flip(pixels[1:,:],axis=1)),axis=0)
    pixels=np.concatenate((pixels,cenStack),axis=0)
    # rotating the quarter circle
    pixels=pixels+0.5
    pixelX=np.multiply(pixels,np.broadcast_to(np.array([-1,1]),(pixels.shape)))
    pixels=np.concatenate((pixels,pixelX),axis=0)
    pixelY=np.multiply(pixels,np.broadcast_to(np.array([1,-1]),(pixels.shape)))
    pixels=np.concatenate((pixels,pixelY),axis=0)
    pixels=pixels-0.5
    return(pixels.astype(int))
def outAround(out,around):
    '''
    Takes 0,0,1 rotates it round the x axis (down) by the out angle and then rotates it around the z axis by the around angle
    only works with one pair of angles at the momen
    '''
    return R.from_euler('xz', [out,around], degrees=True).apply(np.array([0,0,1]))
def pixelCircle(r,centers=False,normalized=False):
    '''
    Returns pixels a distance r from center is at coordiate point 0,0 and pixels are assumed to be 1 unit wide.
    The closest pixel to the center has the center (0.5,0.5) It is assumed that this level of detail will matter some day.
    '''
    r2=r*r
    pixels=np.array([[r*ma.cos(ma.pi/4)-0.5,r*ma.sin(ma.pi/4)-0.5]],dtype=int)
    y = pixels[0,1]
    x = pixels[0,0]
    cenStack=np.stack((np.arange(0,x),np.arange(0,y))).T
    # print(x,y)
    while(y>=0 and x<r):
        # print(x+1,y,ma.pow(x+1+0.5,2)+ma.pow(y+0.5,2))
        if ma.pow(x+1+0.5,2)+ma.pow(y+0.5,2)<r2 :
            x+=1
        else :
            y-=1
        newStack = np.stack((np.arange(x-y,x+1),np.arange(0,y+1))).T
        pixels=np.concatenate((pixels,newStack),axis=0)
        
    # print(np.flip(pixels[1:,:],axis=1))
    pixels=np.concatenate((pixels,np.flip(pixels[1:,:],axis=1)),axis=0)
    pixels=np.concatenate((pixels,cenStack),axis=0)
    # rotating the quarter circle
    pixels=pixels+0.5
    pixelX=np.multiply(pixels,np.broadcast_to(np.array([-1,1]),(pixels.shape)))
    pixels=np.concatenate((pixels,pixelX),axis=0)
    pixelY=np.multiply(pixels,np.broadcast_to(np.array([1,-1]),(pixels.shape)))
    pixels=np.concatenate((pixels,pixelY),axis=0)
    if centers:
        return pixels
    else:
        pixels=pixels-0.5
        return(pixels.astype(int))
def persp2eyeVec(coords):
    '''
    Perspective should refer to the normalized version of what goes onto the screen. Screen should refer to what gets put onto the screen
    Takes perspective coordintes and returns the eye vector corresponding to that perspective coordinate
    '''
    # print(coords)
    coords=np.reshape(coords,(int(coords.size/2),2))
    outDist=np.linalg.norm(coords,axis=1)
    outAngle=np.arcsin(outDist)*2
    aroundAngle=np.arctan2(coords[:,1],coords[:,0])+np.pi/2
    # print(coords.T)
    # print(outDist)
    # next we need to convert the 
    OACoords=np.array([outAngle,aroundAngle]).T
    # print(np.degrees(OACoords).T)
    vector=R.from_euler('xz', OACoords, degrees=False).apply(np.array([0,0,1]))
    return vector

def draw_dot(s,c,l,size=1):
    if isinstance(c,int):
        c=(c,c,c)
    pygame.draw.circle(s,c,l,size)

def draw_dot_screen(s,c,l,size=1):
    if isinstance(c,int):
        c=(c,c,c)
    pygame.draw.circle(s,c,l,size)

In [5]:
# completed number munching
# setting up position
O = np.array([0,0,0])
# setting up some things to look at
fList,cList=octahedron(np.array([0,0,0]),30,R.from_rotvec(np.array([2*np.pi,0,0])))
nf,nc=octahedron(np.array([6,7,8]),5,R.from_rotvec(np.array([10, 15, 5])),highColor=8)
fList,cList = newFace(nf,nc,fList,cList)
fList,cList = newFace(np.array([[[8,0,0],[8,1,0],[8,0,1]]]),np.array([[9,0,0]]),fList,cList)
fList,cList = newFace(np.array([[[0,8,0],[0,8,1],[1,8,0]]]),np.array([[0,9,0]]),fList,cList)
fList,cList = newFace(np.array([[[0,0,8],[1,0,8],[0,1,8]]]),np.array([[0,0,9]]),fList,cList)
fList,cList = newFace(np.array([[[-8,0,0],[-8,1,0],[-8,0,1]]]),np.array([[0,9,9]]),fList,cList)
fList,cList = newFace(np.array([[[0,-8,0],[0,-8,1],[1,-8,0]]]),np.array([[9,0,9]]),fList,cList)
fList,cList = newFace(np.array([[[0,0,-8],[1,0,-8],[0,1,-8]]]),np.array([[9,9,0]]),fList,cList)
fList,cList = newFace(np.array([[[16,0,0],[16,1,0],[16,0,1]]]),np.array([[9,0,0]]),fList,cList)
fList,cList = newFace(np.array([[[0,16,0],[0,16,1],[1,16,0]]]),np.array([[0,9,0]]),fList,cList)
fList,cList = newFace(np.array([[[0,0,16],[1,0,16],[0,1,16]]]),np.array([[0,0,9]]),fList,cList)
cList=cList/9*255
nFaces=fList.shape[0]
# making some sight vectors
perspCoords = pixelCircle(50,True)/50
sightVectors = persp2eyeVec(perspCoords)
# ray tracing
print(perspCoords.shape)
print(sightVectors.shape)

(7860, 2)
(7860, 3)


In [6]:

# font = pygame.font.SysFont(None, 24) # Need to test this module in a smaller environemnt. 

clock = pygame.time.Clock()
SCREEN_DIMENSION=(700,600)
sCx=SCREEN_DIMENSION[0]/2
sCy=SCREEN_DIMENSION[1]/2
sRadius=min(sCx,sCy)
screenCoords = np.multiply(perspCoords*sRadius,np.broadcast_to(np.array([1,-1]),(perspCoords.shape)))+np.array([sCx,sCy]).astype(int)

screen = pygame.display.set_mode(SCREEN_DIMENSION)

player = Player()

# Run until the user asks to quit
running = True
while running:
    # Did the user click the window close button?
    for event in pygame.event.get():
        # Did the user hit a key?
        if event.type == KEYDOWN:
            # Was it the Escape key? If so, stop the loop.
            if event.key == K_ESCAPE:
                running = False
            if event.key == K_ESCAPE:
                running = False
        # Check for QUIT event. If QUIT, then set running to false.
        elif event.type == pygame.QUIT:
            running = False
            
    # Get all the keys currently pressed
    pressed_keys = pygame.key.get_pressed()
    # Update the player based on user keypresses
    player.update(pressed_keys)
    
    fListFlat=fList.reshape(nFaces*3,3)
    fListP=player.orientation.apply(fListFlat).reshape(nFaces,3,3)
    
    # Fill the background with white
    # screen.fill((255, 255, 255))
    
    # sightVectors = R.random(500).apply(pzh)
    nearIndexs=rayNumbersV(sightVectors,player.location,fListP)
    nearColors=cList[nearIndexs]
    # outArnd=np.transpose(np.array(outAroundInv(sightVectors)))
    
    for i in range(0,screenCoords.shape[0]):
        draw_dot_screen(screen,nearColors[i],screenCoords[i],4)

    
    # Draw the reticle
    reticleC=(180, 180, 180)
    pygame.draw.circle(screen, reticleC, (sCx, sCy), sRadius, width=2)
    pygame.draw.circle(screen, reticleC, (sCx, sCy), sRadius*0.707, width=2)
    pygame.draw.line(screen, reticleC, (sCx-sRadius, sCy), (sCx+sRadius, sCy), width=2)
    pygame.draw.line(screen, reticleC, (sCx, sCy-sRadius), (sCx, sCy+sRadius), width=2)
    
    # Flip the display
    pygame.display.flip()
    
    clock.tick(5)

# Done! Time to quit.
pygame.display.quit()

In [ ]:
# Display of location/orientation 
#     learn to draw text. 
# shoot 1 ray per pixel
#     convert screen coordinate too ray